In [132]:
import pandas as pd; import numpy as np; from datetime import timedelta

In [124]:
# Load Data
complaints = pd.read_csv('data/toy.complaint_data.csv'); 
officers = pd.read_csv('data/toy.officer_data.csv'); 

In [165]:
def count_complaints_in_year(year, x):
    x['year'] = pd.DatetimeIndex(x['incident_date']).year 
    x1 = x[(x.year == year)]
    x2 = x1.groupby(['officer_id']).size()
    return x2

def add_complaints_by_year(base_year, nlags, df_complaints, df_officers):
    Y = df_officers
    for year in range(base_year,base_year-nlags,-1):
        q = count_complaints_in_year(year, df_complaints).to_frame()
        j = year-base_year
        jstr = 'Lag%d' % j
        q.columns = [jstr]
        Y = Y.merge(q, left_on = 'officer_id', right_index = 1, how = 'outer')
    Y.fillna(value = 0, inplace = True)
    return Y

def add_lag_to_complaints(df_complaints, scale_days, base_year, base_month, base_day):
    p = timedelta(days=scale_days)
    base_time = datetime(base_year, base_month, base_day);
    incident_time = pd.DatetimeIndex(df_complaints['incident_date'])
    df_complaints['LAG'] = (base_time - incident_time)/p
    df_complaints['LAG'] = df_complaints.LAG.apply(np.floor)
    df_complaints['LAG'] = df_complaints.LAG.apply(int)
    return(df_complaints)
    

In [166]:
# Add lag to complaints data frame.
x = add_lag_to_complaints(complaints, 30, 2016, 1, 1)
x.head()

,crid,incident_date,beat_2012_geocoded,complaintcategory,finalfinding,officer_id,b,LAG
0,1022797,01-Jan-2009 0:08,NaN,03G-MISCELLANEOUS,EX,1,-113.0,85
1,1022797,01-Jan-2009 0:08,NaN,03G-MISCELLANEOUS,EX,2,-113.0,85
2,1022797,01-Jan-2009 0:08,NaN,03G-MISCELLANEOUS,EX,3,-113.0,85
3,1022797,01-Jan-2009 0:08,NaN,03G-MISCELLANEOUS,EX,4,-113.0,85
4,1022797,01-Jan-2009 0:08,NaN,03G-MISCELLANEOUS,EX,5,-113.0,85


#Function 'add_complaints_by_year(base_year, nlags, df_complaints, df_officers)'
Arguments:

base_year = year at which model is being fit

nlags = number of lagged years (relative to base_year) over which officer's complaints are counted.

df_complaints = toy-complaints dataframe

df_officers = toy-officers dataframe


In [ ]:
r = add_complaints_by_year(2015, 12, complaints, officers)

In [26]:
# Identifying worst complaints.
complaints['complaintcategory_short_name'] = complaints['complaintcategory'].str[0:3]

# Identifying worst complaints.
worst_complaints = ["01A","01B","01C","03E","04H","05A","05B","05C","05D","05E","05F","05G","05H","05J","05K","05L","05M","05N","05P","05Q","05T"]
medium_complaints = ["04A","04B","04C","04D","04E","04F","04G","04J"]

complaints['complaint_type'] = complaints.complaintcategory_short_name.apply(lambda x: 2 if x in worst_complaints else 1 if x in medium_complaints else 0)


